In [0]:
import pandas as pd

In [0]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [0]:
dbutils.fs.unmount("/mnt/alv")

/mnt/alv has been unmounted.
Out[3]: True

In [0]:
def mount_blob(storage_account_name, access_key):
  #access_key = dbutils.secrets.get(scope="storage", key=storage_account_name)
  mount_list = [
    {
      "folder_name": "alv",
      "message" : "Mount Azure Blob Storage (alv) in DBFS..."
    }       
  ]
  
  for item in mount_list:
    print(item['message'])    
    folder_name = item['folder_name']
    mount_point = '/mnt/{}'.format(folder_name)
    blob_path = 'wasbs://{}@{}.blob.core.windows.net'.format(folder_name, storage_account_name)
    config = "fs.azure.account.key.{}.blob.core.windows.net".format(storage_account_name)
    try:
      dbutils.fs.ls(mount_point)
      print('{} has been mounted.'.format(mount_point))
    except:
      print('DBFS Mount does not exist, creating mount {}...'.format(mount_point))
      
      dbutils.fs.mount(
        source = blob_path,
        mount_point = mount_point,
        extra_configs = {
          config:access_key
        }
      )

In [0]:
storageAccountName = 'bidseadlg2'
# blobContainerName = 'alv'
storageAccountAccessKey = 'k/GjO0Yn78u+XwP1vaVq2dVaN5spq2YgHmk87K35a6+wt31nsXd4zA2YVaugWsmqcniI511OLHNdYxQZj2nvXQ=='
mount_blob(storageAccountName, storageAccountAccessKey)

Mount Azure Blob Storage (alv) in DBFS...
DBFS Mount does not exist, creating mount /mnt/alv...

In [0]:
def read_data_to_dataframe(file_location):
  file_type = "csv"
  # CSV options
  infer_schema = "false"
  first_row_is_header = "true"
  delimiter = ","
  encoding = 'utf-8'

  # The applied options are for CSV files. For other file types, these will be ignored.
  df = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .option("header", first_row_is_header) \
    .option("sep", delimiter) \
    .option("encoding", encoding) \
    .load(file_location)
  return df

In [0]:
def output_csv_file(df, path):
   (df.coalesce(1) # uncomment this line to ensure that the data is written to just one file
   .write
   .option("header", "true")
   .option("sep", ",")
   .option("encoding", "utf-8")
   .option("ESCAPE quote", '"')
   .mode("overwrite") # overwrite any pre-existing data at the path that we're writing to
   .csv(path)
  )

In [0]:
file_list = []
df_list = []
folder_path = '/mnt/alv/'
files = dbutils.fs.ls(folder_path + 'TEST')

for f in files:
  file_list.append(f.name)
  print(file_list)
  df = read_data_to_dataframe(f.path).toPandas()
  df = df.rename(columns={df.columns[len(df.columns)-1]: 'SourceFile'})
  df_list.append(df)

print(df.head())

['TEST_CN20 _201901.csv']
['TEST_CN20 _201901.csv', 'TEST_CN20_201902.csv']
 工廠 申請單號 項次 報廢狀態 ... 除帳物料文件 除帳數量 除帳內文 SourceFile
0 CN20 A180922021 1 除帳 ... 4901310555 950 版本升級 TEST_CN20_201902.csv
1 CN20 A180922021 2 除帳 ... 4901310555 1000 版本升級 TEST_CN20_201902.csv
2 CN20 A180922021 3 除帳 ... 4901310555 5670 版本升級 TEST_CN20_201902.csv
3 CN20 A180922021 4 除帳 ... 4901310555 150 版本升級 TEST_CN20_201902.csv
4 CN20 A180922021 5 除帳 ... 4901310555 300 版本升級 TEST_CN20_201902.csv

[5 rows x 36 columns]

In [0]:
df_combine = pd.concat(df_list)
df_combine = df_combine.dropna(axis = 1, how = 'all')
df_combine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 0 to 8
Data columns (total 31 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 工廠 18 non-null object
 1 申請單號 18 non-null object
 2 項次 18 non-null object
 3 報廢狀態 18 non-null object
 4 申請人員 18 non-null object
 5 成本中心 18 non-null object
 6 客戶代號 18 non-null object
 7 客戶名稱 18 non-null object
 8 物料 18 non-null object
 9 物料說明 18 non-null object
 10 批號 18 non-null object
 11 移倉物料文件 18 non-null object
 12 數量 18 non-null object
 13 SKU 18 non-null object
 14 每單位量 18 non-null object
 15 報廢原因 18 non-null object
 16 原因說明 18 non-null object
 17 幣別22 18 non-null object
 18 報廢金額(成本) 18 non-null object
 19 CNY報廢金額(成本) 18 non-null object
 20 結算金額 18 non-null object
 21 CNY結算金額 18 non-null object
 22 幣別27 18 non-null object
 23 報廢金額(銷售) 18 non-null object
 24 幣別29 18 non-null object
 25 本國貨幣金額 18 non-null object
 26 過帳日期 18 non-null object
 27 除帳物料文件 18 non-null object
 28 除帳數量 18 non-null object
 29 除帳內文 18 non-null object
 30 SourceFile 18 non-null object
dtypes: object(31)
memory usage: 4.5+ KB

In [0]:
# pandas dataframe to spark dataframe
df_spark = spark.createDataFrame(df_combine)
display(df_spark)

工廠,申請單號,項次,報廢狀態,申請人員,成本中心,客戶代號,客戶名稱,物料,物料說明,批號,移倉物料文件,數量,SKU,每單位量,報廢原因,原因說明,幣別22,報廢金額(成本),CNY報廢金額(成本),結算金額,CNY結算金額,幣別27,報廢金額(銷售),幣別29,本國貨幣金額,過帳日期,除帳物料文件,除帳數量,除帳內文,SourceFile
CN20,A180922021,1,除帳,張蒙,生管部,30118603,南寧富桂精密工業有限公司,A000027231A,AW-12184-04,C184070204,4901297210,950,PC,0,3506,超印(基本上機耗損),CNY,320.58,320.58,580.64,580.64,CNY,409.45,CNY,409.45,1/30/2019,4901310555,950,版本升級,TEST_CN20_201903.csv
CN20,A180922021,2,除帳,張蒙,生管部,30118603,南寧富桂精密工業有限公司,A000027357A,AW-13414-01,C184070223,4901297210,1000,PC,0,3506,超印(基本上機耗損),CNY,362.38,362.38,597.61,597.61,CNY,591,CNY,591,1/30/2019,4901310555,1000,版本升級,TEST_CN20_201903.csv
CN20,A180922021,3,除帳,張蒙,生管部,30118603,南寧富桂精密工業有限公司,A000027233A,56-1000021-02,C184090017,4901297210,5670,PC,0,3506,超印(基本上機耗損),CNY,2844.36,2844.36,3519.22,3519.22,CNY,3158.76,CNY,3158.76,1/30/2019,4901310555,5670,版本升級,TEST_CN20_201903.csv
CN20,A180922021,4,除帳,張蒙,生管部,30118603,南寧富桂精密工業有限公司,A000049806A,B500.02259.005,C184090641,4901297210,150,PC,0,3506,超印(基本上機耗損),CNY,69.08,69.08,119.17,119.17,CNY,119.17,CNY,119.17,1/30/2019,4901310555,150,版本升級,TEST_CN20_201903.csv
CN20,A180922021,5,除帳,張蒙,生管部,30118603,南寧富桂精密工業有限公司,A000027356A,AW-13416-01,C184130528,4901297210,300,PC,0,3506,超印(基本上機耗損),CNY,108.71,108.71,457.45,457.45,CNY,177.3,CNY,177.3,1/30/2019,4901310555,300,版本升級,TEST_CN20_201903.csv
CN20,A180922021,6,除帳,張蒙,生管部,30118603,南寧富桂精密工業有限公司,A000027373A,56-1001588-01,C184131451,4901297210,60,PC,0,3506,超印(基本上機耗損),CNY,177.01,177.01,200.91,200.91,CNY,138.2,CNY,138.2,1/30/2019,4901310555,60,版本升級,TEST_CN20_201903.csv
CN20,A180922021,7,除帳,張蒙,生管部,301030,昊陽天宇科技(深圳)有限公司,A000036011A,"520-069676 REV. 03,有升级04版",C184190140,4901297210,260,PC,0,3506,超印(基本上機耗損),CNY,60.53,60.53,61.74,61.74,CNY,205.71,CNY,205.71,1/30/2019,4901310555,260,版本升級作廢,TEST_CN20_201903.csv
CN20,A180922021,8,除帳,張蒙,生管部,301030,昊陽天宇科技(深圳)有限公司,A000052254A,246000001984A REV.01,C184200045,4901297210,600,PC,0,3506,超印(基本上機耗損),CNY,128.92,128.92,146.17,146.17,CNY,229.32,CNY,229.32,1/30/2019,4901310555,600,版本升級作廢,TEST_CN20_201903.csv
CN20,A180922021,9,除帳,張蒙,生管部,301030,昊陽天宇科技(深圳)有限公司,A000052255A,246000000817A REV.06,C184200052,4901297210,99,PC,0,3506,超印(基本上機耗損),CNY,25.36,25.36,27.32,27.32,CNY,17.39,CNY,17.39,1/30/2019,4901310555,99,版本升級作廢,TEST_CN20_201901.csv
CN20,A180922021,1,除帳,張蒙,生管部,30118603,南寧富桂精密工業有限公司,A000027231A,AW-12184-04,C184070204,4901297210,950,PC,0,3506,超印(基本上機耗損),CNY,320.58,320.58,580.64,580.64,CNY,409.45,CNY,409.45,1/30/2019,4901310555,950,版本升級,TEST_CN20_201902.csv


In [0]:
write_path = "{}/TEST_merge".format(folder_path)
output_csv_file(df_spark, write_path)

tmp_files = dbutils.fs.ls(write_path)
output_file = [x for x in tmp_files if x.name.startswith("part-")]

# Move the wrangled-data CSV file from a sub-folder (wrangled_data_folder) to the root of the blob container
# While simultaneously changing the file name
dbutils.fs.mv(output_file[0].path, "{}/TEST.csv".format(folder_path))
dbutils.fs.rm(write_path, recurse = True)

Out[34]: True